In [2]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
with_mask = np.load("with_mask.npy")
without_mask = np.load("without_mask.npy")

In [4]:
print(with_mask.shape)
print(without_mask.shape)

(200, 50, 50, 3)
(200, 50, 50, 3)


In [5]:
with_mask = with_mask.reshape(200, 50*50*3)
without_mask = without_mask.reshape(200, 50*50*3)

In [6]:
X = np.r_[with_mask, without_mask] #
labels = np.zeros(X.shape[0])
print(X," ", len(X))

[[255   0 255 ...  43  43  43]
 [255   0 255 ... 119  57 119]
 [251   6 251 ...  28  33  39]
 ...
 [255   0 255 ... 220  32 221]
 [255   0 255 ... 223  29 225]
 [255   0 255 ... 225  28 227]]   400


In [7]:
labels[200:] = 1.0 # bắt đầu từ ảnh thứ 201 có nhãn dán = 1 (ảnh without_mask)

In [8]:
# khai báo nhãn dãn
names = {0: "mask", 1: "no mask"}

In [9]:
# chia dữ liệu
x_train,x_test,y_train,y_test = train_test_split(X,labels,test_size=0.25)
x_train, x_validation, y_train, y_validation=train_test_split(x_train, y_train, test_size=0.25)

In [10]:
# build model SVM
svm = SVC()
svm.fit(x_train,y_train)

SVC()

In [11]:
y_pred = svm.predict(x_test)

In [ ]:
# đoạn này tương tự cái test.inpy nhé
haar_data = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
font=cv2.FONT_HERSHEY_COMPLEX
capture = cv2.VideoCapture(0)
data = [] #save data
while True:
    flag, img = capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img) 
        for x,y,w,h in faces:
            face = img[y:y+h,x:x+w,:]
            face = cv2.resize(face,(50,50))
            face = face.reshape(1,-1) #đưa về 1 mảng 1 chiều
            pred  = svm.predict(face)[0]
            print(pred, " ", pred[0])
            n = names[int(pred)
            if(n == "no mask"):
                cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
                cv2.rectangle(img, (x,y-40),(x+w, y), (0,255,0),-2)
               
                cv2.putText(img,"no mask", (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(255,255,255),1, cv2.LINE_AA)
            else:
                cv2.rectangle(img,(x,y),(x+w,y+h),(50,50,255),2)
                cv2.rectangle(img, (x,y-40),(x+w, y), (50,50,255),-2)
                cv2.putText(img,"mask", (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(255,255,255),1, cv2.LINE_AA)
                
                            
        cv2.imshow("result", img) # show image   
        if cv2.waitKey(2) == 27 or len(data)>=200:
            break
            
capture.release()
cv2.destroyAllWindows()